In [1]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
def generate_weather_data(days=365*5):
    dates = pd.date_range(start='2010-01-01', periods=days)
    data = {
        'temp': np.sin(2 * np.pi * np.arange(days)/365) * 20 + 25 + np.random.normal(0, 3, days),
        'humidity': np.cos(2 * np.pi * np.arange(days)/365) * 20 + 50 + np.random.normal(0, 5, days),
        'pressure': np.random.normal(1013, 10, days),
        'precipitation': np.random.exponential(0.1, days)
    }
    df = pd.DataFrame(data, index=dates)
    return df

In [3]:
weather_df = generate_weather_data()
features = ['temp', 'humidity', 'pressure', 'precipitation']


In [4]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(weather_df[features])

In [5]:
def create_sequences(data, n_steps):
    X, y = [], []
    for i in range(len(data)-n_steps-1):
        X.append(data[i:(i+n_steps), :])
        y.append(data[i+n_steps, 0])  # Predict temperature
    return np.array(X), np.array(y)

n_steps = 30  # Use 30 days to predict next day
X, y = create_sequences(scaled_data, n_steps)

In [6]:
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

In [7]:
model = Sequential ([

    LSTM(100, return_sequences=True, input_shape=(n_steps, X_train.shape[2])),
    Dropout(0.3),
    LSTM(50, return_sequences=False)),
    Dropout(0.3),
    Dense(25, activation='relu'),
    Dense(1) 
])


SyntaxError: closing parenthesis ')' does not match opening parenthesis '[' on line 1 (197742223.py, line 5)

In [ ]:
model.compile(optimizer='adam',loss='mse', metrics=['mae'])
early_stop = EarlyStopping(monitor='val_loss', patience=10)


In [ ]:
history = model.fit(X_train, y_train,
                    epochs=100,
                    batch_size=64,
                    validation_split=0.2,
                    callbacks=[early_stop],
                    verbose=1)

In [ ]:
test_predictions = model.predict(X_test)
test_predictions = scaler.inverse_transform(
    np.concatenate((test_predictions, np.zeros((len(test_predictions), len(features)-1))), axis=1)
)[:, 0]

y_test_actual = scaler.inverse_transform(
    np.concatenate((y_test.reshape(-1,1), np.zeros((len(y_test), len(features)-1))), axis=1)
)[:, 0]

In [ ]:
plt.figure(figsize=(18, 12))

In [ ]:
plt.subplot(2, 2, 1)
sns.heatmap(weather_df.corr(), annot=True, cmap='coolwarm')
plt.title('Feature Correlation Matrix')

In [ ]:
plt.subplot(2, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training History')
plt.legend()

In [ ]:
plt.subplot(2, 2, 3)
plt.plot(y_test_actual, label='Actual Temperature')
plt.plot(test_predictions, label='Predicted Temperature')
plt.title('Temperature Predictions')
plt.legend()